In [1]:
from datasets import load_dataset
import pandas as pd
import re, unicodedata

In [2]:
ds = load_dataset("nyuuzyou/wb-products", split="train", streaming=True)

filtered = ds.filter(lambda x: 5 <= len(x["description"]) <= 100)

subset = filtered.take(515_000)

subset_list = list(subset)

data = pd.DataFrame(subset_list)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
def preprocess_text(s: str, punct_to="space", lower=False, keep: str = ""):
    """
    punct_to: 'space' -> пунктуацию/символы заменить на пробел; 'drop' -> удалить.
    keep: строка символов, которые НЕ трогаем (например, '-' или '+'), по умолчанию не сохраняем ничего.
    """
    if not isinstance(s, str):
        s = "" if s is None else str(s)

    # NFC-нормализация и унификация пробелов
    s = unicodedata.normalize("NFC", s)
    s = s.replace("\u00A0", " ").replace("\u2009", " ").replace("\u202F", " ")

    out = []
    for ch in s:
        cat = unicodedata.category(ch)
        if ch in keep:
            out.append(ch)
        elif cat.startswith('Z'):
            out.append(' ')
        elif cat.startswith(('P','S')):
            if punct_to == "space":
                out.append(' ')
        else:
            out.append(ch)

    s = "".join(out)
    s = re.sub(r"\s+", " ", s).strip()
    if lower:
        s = s.lower()
    return s

def compact_and_boundaries_from_clean(s: str):
    """
    На вход подаём уже очищенную строку.
    Возвращает:
      no_space — строка без пробелов;
      pos      — позиции в координатах no_space, после которых в ИСХОДНОЙ очищенной строке был >=1 пробел.
    """
    out = []
    pos = []
    in_word = False

    for ch in s:
        if ch == ' ':
            if in_word and out:
                pos.append(len(out) - 1)
            in_word = False
            continue
        out.append(ch)
        in_word = True

    return "".join(out), pos

In [4]:
prepared_rows = []
for i, row in data.iterrows():
    clean = preprocess_text(row["description"], punct_to="space", lower=False, keep="")
    no_space, positions = compact_and_boundaries_from_clean(clean)
    prepared_rows.append({
        "id": i,
        "input_text": no_space,
        "target_positions": " ".join(map(str, positions)),
    })

df_prepared = pd.DataFrame(prepared_rows, columns=["id","input_text","target_positions"])

In [5]:
s = "Юбка LAFEI-NIER из качественного хлопка, разрез со спины. Карманы декорированы молниями."
clean = preprocess_text(s, punct_to="space", keep="")
x, y = compact_and_boundaries_from_clean(clean)
print(x)  # КлассическийприталенныйпиджакизстрейчтканиБоль
print(y)  # [11, 22, 28, 30, 36, 41]

ЮбкаLAFEINIERизкачественногохлопкаразрезсоспиныКарманыдекорированымолниями
[3, 8, 12, 14, 27, 33, 39, 41, 46, 53, 65]


In [6]:
# Удаляем строки с одинаковым input_text
df_prepared = df_prepared.drop_duplicates(subset=["input_text"]).reset_index(drop=True)

print("После удаления дубликатов:", len(df_prepared))

После удаления дубликатов: 208349


In [7]:
df_prepared.head()

,id,input_text,target_positions
0,0,Наузкующиколотки,1 6
1,1,Бриджистрейчсажуромизхлопка,5 11 12 18 20
2,3,КлассическийприталенныйпиджакизстрейчтканиБоль...,11 22 28 30 36 48
3,4,ЮбкаLAFEINIERизкачественногохлопкаразрезсоспин...,3 8 12 14 27 33 39 41 46 53 65
4,5,МониторACERNitroVG270bmipxIPS1920x108075HzUMHV...,6 10 15 25 28 37 41 43 48


In [8]:
df_prepared.to_csv("train_dataset.csv", index=False)